In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
import re
from gensim.models.doc2vec import Doc2Vec , TaggedDocument


In [2]:
depressed_dataset = pd.read_csv("D:\Workspace\OVGU\SM_Depression\src\depTweetsOrderedByDateTimeWindowURLTokens.csv")

In [3]:
depressed_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61972 entries, 0 to 61971
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   accountname       61972 non-null  object 
 1   description       61483 non-null  object 
 2   favorite_count    61937 non-null  float64
 3   location          51551 non-null  object 
 4   masked_user_id    61972 non-null  object 
 5   retweet_count     61968 non-null  object 
 6   timestamp         61972 non-null  object 
 7   tweets            61972 non-null  object 
 8   twittername       61972 non-null  object 
 9   user_type         61972 non-null  float64
 10  format            61972 non-null  int64  
 11  new_date          61972 non-null  object 
 12  time_window       61972 non-null  int64  
 13  tweets_url_token  61972 non-null  object 
 14  cleanData         61728 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 7.1+ MB


In [4]:
print(tf.__version__)

2.2.0


In [5]:
depressed_dataset.head()

,accountname,description,favorite_count,location,masked_user_id,retweet_count,timestamp,tweets,twittername,user_type,format,new_date,time_window,tweets_url_token,cleanData
0,Cmdr. Nasty Tinkerbell of the 4077th,NSFW \r\r\r\r\nWoman \r\r\r\r\nDisabled \r\r\r...,0.0,Exploring the universe,depressed_user_11,0.0,24-06-2020 00:00,@Twitter it's about time.\r\r\r\r\n\r\r\r\r\nL...,CmdrTinkerBell,1.0,1,2020-06-24 00:00,1,@Twitter it's about time.\r\r\r\r\n\r\r\r\r\nL...,it's about time. Lol IT'S NOT FAIR [URL] [URL]
1,Cmdr. Nasty Tinkerbell of the 4077th,NSFW \r\r\r\r\nWoman \r\r\r\r\nDisabled \r\r\r...,0.0,Exploring the universe,depressed_user_11,0.0,24-06-2020 00:02,"@w_terrence @PearlCathey8 Like deplorable, lik...",CmdrTinkerBell,1.0,1,2020-06-24 00:02,1,"@w_terrence @PearlCathey8 Like deplorable, lik...","Like deplorable, like donnie, have to use cont..."
2,Cmdr. Nasty Tinkerbell of the 4077th,NSFW \r\r\r\r\nWoman \r\r\r\r\nDisabled \r\r\r...,0.0,Exploring the universe,depressed_user_11,0.0,24-06-2020 00:05,@HarleyQuinnlif3 Dick always ruin things lol,CmdrTinkerBell,1.0,1,2020-06-24 00:05,1,@HarleyQuinnlif3 Dick always ruin things lol,Dick always ruin things lol
3,Cmdr. Nasty Tinkerbell of the 4077th,NSFW \r\r\r\r\nWoman \r\r\r\r\nDisabled \r\r\r...,2.0,Exploring the universe,depressed_user_11,0.0,24-06-2020 00:13,After seeing pics of todays rally in Phoenix I...,CmdrTinkerBell,1.0,1,2020-06-24 00:13,1,After seeing pics of todays rally in Phoenix I...,After seeing pics of todays rally in Phoenix I...
4,Cmdr. Nasty Tinkerbell of the 4077th,NSFW \r\r\r\r\nWoman \r\r\r\r\nDisabled \r\r\r...,2.0,Exploring the universe,depressed_user_11,0.0,24-06-2020 00:13,if I do get covid19 becuase I'm surrounded by ...,CmdrTinkerBell,1.0,1,2020-06-24 00:13,1,if I do get covid19 becuase I'm surrounded by ...,if I do get covid19 becuase I'm surrounded by ...


In [6]:
print(depressed_dataset.isnull().sum())


depressed_dataset = depressed_dataset.replace(np.nan, '', regex=True)

print(depressed_dataset.isnull().sum())
print("")
# print(depressed_dataset.info())

accountname             0
description           489
favorite_count         35
location            10421
masked_user_id          0
retweet_count           4
timestamp               0
tweets                  0
twittername             0
user_type               0
format                  0
new_date                0
time_window             0
tweets_url_token        0
cleanData             244
dtype: int64
accountname         0
description         0
favorite_count      0
location            0
masked_user_id      0
retweet_count       0
timestamp           0
tweets              0
twittername         0
user_type           0
format              0
new_date            0
time_window         0
tweets_url_token    0
cleanData           0
dtype: int64



In [7]:
tweet_text = depressed_dataset.loc[:,'cleanData']

In [8]:
tweet_text

0           it's about time. Lol IT'S NOT FAIR [URL] [URL]
1        Like deplorable, like donnie, have to use cont...
2                              Dick always ruin things lol
3        After seeing pics of todays rally in Phoenix I...
4        if I do get covid19 becuase I'm surrounded by ...
                               ...                        
61967    You need not to be afraid anymore. You are lov...
61968    I did register but I still didn't get the link...
61969               You have a beautiful name too! ð???â?�
61970                                                 yes!
61971                                 Can I get the link??
Name: cleanData, Length: 61972, dtype: object

In [9]:
def preprocessing(String): #default value is always true for stemming and stopwords
#     print(String)
#     print("")
    '''
    This function is used for preprocessing
    - Tokenization
    - Stemming
    - Stop Words
        
    '''
    
    influential_words = re.sub(r'[^a-zA-Z0-9_\[\]]', ' ', String)
#     print(influential_words)
#     tokens = nltk.word_tokenize(String)
#     print(tokens)
#     token = [word for word in tokens if word.isalpha()]
#     print(token)
#     influential_words = " ".join(token)
#     print(influential_words)
    influential_words = influential_words.lower()
#     print(influential_words)
#     influential_words = influential_words.split()
#     print(influential_words)
#     stemwords_string = " ".join(influential_words)
#     print(stemwords_string)
    return influential_words

In [10]:
final_text = []
for i in range(0,len(depressed_dataset.axes[0])):
    final_text.append(str(preprocessing(tweet_text[i])))

In [11]:
depressed_dataset['cleanData'] = final_text

In [12]:
del final_text,tweet_text,i

In [13]:
#clean special characters from cleanData row to apply BERT embeddings
def preprocess(tweets):
    tweets = tweets.replace("(<br/>)", "")
    tweets = tweets.replace('(<a).*(>).*(</a>)', '')
    tweets = tweets.replace('(&amp)', '')
    tweets = tweets.replace('(&gt)', '')
    tweets = tweets.replace('(&lt)', '')
    tweets = tweets.replace('(â€™)', '')
    tweets = tweets.replace('(\xa0)', ' ')
    tweets = tweets.replace('(https)', ' ')
    tweets = tweets.replace('(RT)', ' ')
    tweets = tweets.replace('(ðÿ)', '  ')
    tweets = tweets.replace('(\n)', '  ')
    return tweets

In [14]:
depressed_dataset['cleanData'] = depressed_dataset['cleanData'].apply(preprocess).astype(str)


In [15]:
import sys

In [16]:
depressed_dataset_1 = depressed_dataset[depressed_dataset.time_window == 1]
depressed_dataset_2 = depressed_dataset[depressed_dataset.time_window == 2]
depressed_dataset_3 = depressed_dataset[depressed_dataset.time_window == 3]
depressed_dataset_4 = depressed_dataset[depressed_dataset.time_window == 4]
depressed_dataset_5 = depressed_dataset[depressed_dataset.time_window == 5]
depressed_dataset_6 = depressed_dataset[depressed_dataset.time_window == 6]
depressed_dataset_7 = depressed_dataset[depressed_dataset.time_window == 7]
depressed_dataset_8 = depressed_dataset[depressed_dataset.time_window == 8]
depressed_dataset_9 = depressed_dataset[depressed_dataset.time_window == 9]
depressed_dataset_10 = depressed_dataset[depressed_dataset.time_window == 10]
depressed_dataset_11 = depressed_dataset[depressed_dataset.time_window == 11]
depressed_dataset_12 = depressed_dataset[depressed_dataset.time_window == 12]
depressed_dataset_13 = depressed_dataset[depressed_dataset.time_window == 13]
depressed_dataset_14 = depressed_dataset[depressed_dataset.time_window == 14]


for i in range(1,15):
    filename = "depressed_dataset_"+str(i)
    print(filename)
    print(getattr(sys.modules[__name__], f"depressed_dataset_{i}").shape)
    print("")

depressed_dataset_1
(700, 15)

depressed_dataset_2
(1093, 15)

depressed_dataset_3
(3223, 15)

depressed_dataset_4
(4868, 15)

depressed_dataset_5
(5463, 15)

depressed_dataset_6
(6878, 15)

depressed_dataset_7
(7786, 15)

depressed_dataset_8
(8278, 15)

depressed_dataset_9
(8394, 15)

depressed_dataset_10
(5696, 15)

depressed_dataset_11
(4408, 15)

depressed_dataset_12
(2729, 15)

depressed_dataset_13
(1569, 15)

depressed_dataset_14
(887, 15)



In [17]:
model = Doc2Vec.load('./saved_models/doc2vec_new_300_5_v2/kaggleTune_500_3.d2v')


In [23]:
import pickle
import os
from collections import OrderedDict 
from nltk.tokenize import word_tokenize

embed_path = r"D:\Workspace\OVGU\SM_Depression\scripts\doc2vec_embeddings"
os.chdir(embed_path)

def get_store_embeddings(dataframe,dataframe_name):
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(dataframe['cleanData'])]
    
    inferred_vectors = [model.infer_vector(doc.words, steps=20) for doc in tagged_data]
    
    print(len(dataframe['cleanData']))
    print(len(inferred_vectors))
    print(len(tagged_data))
    embed_name = dataframe_name +"_embedding.pkl"
    print(embed_name)
#     for sentence, embedding in zip(dataframe.cleanData.tolist(), embeddings_array):
#         print("Sentence:", sentence)
#         print("Embedding:", embedding)
#         print(embedding.shape)
#         print("")
    #Store sentences & embeddings on disc
    with open(embed_name, "wb") as fOut:
        pickle.dump(OrderedDict(zip(dataframe.cleanData.tolist(), inferred_vectors)), fOut, protocol=pickle.HIGHEST_PROTOCOL)
#     return embeddings_array

In [24]:
%%time

get_store_embeddings(depressed_dataset_1,"depressed_dataset_1")
get_store_embeddings(depressed_dataset_2,"depressed_dataset_2")
get_store_embeddings(depressed_dataset_3,"depressed_dataset_3")
get_store_embeddings(depressed_dataset_4,"depressed_dataset_4")
get_store_embeddings(depressed_dataset_5,"depressed_dataset_5")
get_store_embeddings(depressed_dataset_6,"depressed_dataset_6")
get_store_embeddings(depressed_dataset_7,"depressed_dataset_7")
get_store_embeddings(depressed_dataset_8,"depressed_dataset_8")
get_store_embeddings(depressed_dataset_9,"depressed_dataset_9")
get_store_embeddings(depressed_dataset_10,"depressed_dataset_10")
get_store_embeddings(depressed_dataset_11,"depressed_dataset_11")
get_store_embeddings(depressed_dataset_12,"depressed_dataset_12")
get_store_embeddings(depressed_dataset_13,"depressed_dataset_13")
get_store_embeddings(depressed_dataset_14,"depressed_dataset_14")

700
700
700
depressed_dataset_1_embedding.pkl
1093
1093
1093
depressed_dataset_2_embedding.pkl
3223
3223
3223
depressed_dataset_3_embedding.pkl
4868
4868
4868
depressed_dataset_4_embedding.pkl
5463
5463
5463
depressed_dataset_5_embedding.pkl
6878
6878
6878
depressed_dataset_6_embedding.pkl
7786
7786
7786
depressed_dataset_7_embedding.pkl
8278
8278
8278
depressed_dataset_8_embedding.pkl
8394
8394
8394
depressed_dataset_9_embedding.pkl
5696
5696
5696
depressed_dataset_10_embedding.pkl
4408
4408
4408
depressed_dataset_11_embedding.pkl
2729
2729
2729
depressed_dataset_12_embedding.pkl
1569
1569
1569
depressed_dataset_13_embedding.pkl
887
887
887
depressed_dataset_14_embedding.pkl
Wall time: 1min 5s
